In [33]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd
import random

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [34]:
EMPTY = 0
BOX = 1
ROBOT = 3
MAX_BOX_STACK = 5

SEED = 67890

In [35]:
class StorageRobot(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.carried_box = False
        self.random.seed(SEED)

    def step(self):
        self.move()
        self.interact_with_box()

    def move(self):

        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
        empty_moves = [move for move in possible_moves if self.model.is_cell_empty_and_no_boxes(move[0], move[1])]

        if empty_moves:
            next_move = self.random.choice(empty_moves)
            self.model.grid.move_agent(self, next_move)

    def interact_with_box(self):
        if not self.carried_box:
            self.pick_up_box()
        else:
            self.stack_box()

    def pick_up_box(self):
        adjacent_cells = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
        for cell in adjacent_cells:
            x, y = cell
            # Verifica si hay una caja en la celda adyacente
            if self.model.boxes[x, y] > 0:
                #Recoje la caja
                self.model.boxes[x, y] -= 1
                self.carried_box = True
                break

    def stack_box(self):
        adjacent_cells = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
        for cell in adjacent_cells:
            x, y = cell
            # Verifica si puede apilar la caja aquí
            if self.can_stack_box(x, y):
                # Apila la caja
                self.model.boxes[x, y] += 1
                self.carried_box = False
                break

    def can_stack_box(self, x, y):
        # Verifica si la celda no tiene el máximo de cajas y no está ocupada por otro robot
        return self.model.boxes[x, y] < MAX_BOX_STACK and self.model.is_cell_empty_and_no_boxes(x, y)


In [36]:

class StorageModel(Model):
    def __init__(self, width, height, n_boxes, n_robots):
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)
        self.boxes = np.zeros((width, height))
        self.generate_boxes(n_boxes)
        self.place_robots(n_robots)
        self.running = True
        self.random.seed(SEED)

    def step(self):
        self.schedule.step()
        if self.all_boxes_stacked():
            self.running = False

    def generate_boxes(self, n_boxes):
        random.seed(SEED)
        placed_boxes = 0

        while placed_boxes < n_boxes:
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)

            boxes_to_place = min(random.randint(1, 3), n_boxes - placed_boxes)

            if self.boxes[x, y] == 0 and boxes_to_place <= 3:
                self.boxes[x, y] = boxes_to_place
                placed_boxes += boxes_to_place

    def place_robots(self, n_robots):
        random.seed(SEED)
        placed_robots = 0

        while placed_robots < n_robots:
            
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)

            if self.is_cell_empty_and_no_boxes(x, y):
                robot = StorageRobot(placed_robots, self)
                self.grid.place_agent(robot, (x, y))
                self.schedule.add(robot)
                placed_robots += 1

    def is_cell_empty_and_no_boxes(self, x , y):
        # Verifica si la celda no tiene cajas ni robots
        cell_contents = self.grid.get_cell_list_contents([(x, y)])
        return len(cell_contents) == 0 and self.boxes[x ,y] == 0
    
    def all_boxes_stacked(self):
        for (x, y), box_count in np.ndenumerate(self.boxes):
            if box_count > 0 and box_count < MAX_BOX_STACK:
                return False
        return True

In [37]:
HEIGHT = 20
WIDTH = 20
MAX_BOXES = 200
MAX_ROBOTS = 5


model = StorageModel(HEIGHT, WIDTH, MAX_BOXES, MAX_ROBOTS)

# Inicializa un contador de pasos
step_count = 0

# Ejecuta la simulación hasta que se complete la condición de finalización
while model.running:
    model.step()
    step_count += 1
    # print(step_count)

# Imprime el número total de pasos al final de la simulación
print(f"La simulación se completó en {step_count} pasos.")


KeyboardInterrupt: 